In [2]:
spark

In [3]:
sc

<SparkContext master=spark://devenv:7077 appName=PySparkShell>

# 0. Import Data and Combine features

In [47]:
# Prepare data
raw = spark.read.csv("hdfs://devenv/user/spark/spark_mllib_101/company/",
                      inferSchema=True,
                      header=True)

In [48]:
raw.show(10, truncate=True)

+-----+--------------------+---------------+-------------------+------------+------------+------------+-------------------------+------------------+------------+------------+----------------+-----------------+------------------------+----------------+---------------+-------------+-----------------+-----------------+------------+------------+------------+------------+-------------+---------------+---------------+----------------------+----------------------+--------------------------+--------------------------+-----------------------------+------------------------+------------------------+------------------------+---------------------+---------------------+-------------------------+-------------------------+----------------------------+-----------------------+-----------------------+-----------------------+----------------------+----------------------+--------------------------+--------------------------+-----------------------------+------------------------+------------------------+---

In [49]:
data = raw.select('label',
                    'roa_rate',
                    'roe_rate',
                    'cash_reinvest_rate',
                    'foreign_rate_bys',
                    'revenue_growth_rate',
                    'net_profit_rate',
                    'operating_gross_rate',
                    'current_rate',
                    'export_kgm_weight_37050000306',
                    'quick_rate',
                    'avg_export_rate',
                    'avg_import_rate',
                    'new_cases_smoothed_USA',
                    'new_cases_smoothed_TWN',
                    'new_cases_smoothed_OWID_EUR',
                    'avgas_cpc',
                    'export_kgm_weight_37079090',
                    'avg_board_bys',
                    'people_fully_vaccinated_per_hundred_USA',
                    'import_kgm_weight_37071000',
                    'change_rate_bys',
                    'export_kgm_weight_2804',
                    'export_usd_value_2804',
                    'export_usd_value_848620',
                    'import_usd_value_37079090',
                    'import_kgm_weight_848610',
                    'export_usd_value_37079090',
                    'export_kgm_weight_2801',
                    'under400_rate_bys',
                    'debt_rate',
                    'N1NG',
                    'export_usd_value_2801',
                    'export_kgm_weight_848610',
                    'oil_avg_value',
                    'import_usd_value_848610',
                    'import_kgm_weight_381800',
                    'export_kgm_weight_381800',
                    'import_usd_value_2801',
                    'import_usd_value_381800',)



In [50]:
data.show(10)

+-----+------------+------------+------------------+----------------+-------------------+---------------+--------------------+------------+-----------------------------+------------+---------------+---------------+----------------------+----------------------+---------------------------+------------+--------------------------+-------------+---------------------------------------+--------------------------+---------------+----------------------+---------------------+-----------------------+-------------------------+------------------------+-------------------------+----------------------+-----------------+------------+------------+---------------------+------------------------+-------------+-----------------------+------------------------+------------------------+---------------------+-----------------------+
|label|    roa_rate|    roe_rate|cash_reinvest_rate|foreign_rate_bys|revenue_growth_rate|net_profit_rate|operating_gross_rate|current_rate|export_kgm_weight_37050000306|  quick_rat

In [51]:
data.dtypes

[('label', 'int'),
 ('roa_rate', 'double'),
 ('roe_rate', 'double'),
 ('cash_reinvest_rate', 'double'),
 ('foreign_rate_bys', 'double'),
 ('revenue_growth_rate', 'double'),
 ('net_profit_rate', 'double'),
 ('operating_gross_rate', 'double'),
 ('current_rate', 'double'),
 ('export_kgm_weight_37050000306', 'double'),
 ('quick_rate', 'double'),
 ('avg_export_rate', 'double'),
 ('avg_import_rate', 'double'),
 ('new_cases_smoothed_USA', 'double'),
 ('new_cases_smoothed_TWN', 'double'),
 ('new_cases_smoothed_OWID_EUR', 'double'),
 ('avgas_cpc', 'double'),
 ('export_kgm_weight_37079090', 'double'),
 ('avg_board_bys', 'double'),
 ('people_fully_vaccinated_per_hundred_USA', 'double'),
 ('import_kgm_weight_37071000', 'double'),
 ('change_rate_bys', 'double'),
 ('export_kgm_weight_2804', 'double'),
 ('export_usd_value_2804', 'double'),
 ('export_usd_value_848620', 'double'),
 ('import_usd_value_37079090', 'double'),
 ('import_kgm_weight_848610', 'double'),
 ('export_usd_value_37079090', 'double')

In [52]:
numCols = [x for (x, dataType) in data.dtypes if ((dataType == 'double')&(x != 'label'))]

In [53]:
print(numCols)

['roa_rate', 'roe_rate', 'cash_reinvest_rate', 'foreign_rate_bys', 'revenue_growth_rate', 'net_profit_rate', 'operating_gross_rate', 'current_rate', 'export_kgm_weight_37050000306', 'quick_rate', 'avg_export_rate', 'avg_import_rate', 'new_cases_smoothed_USA', 'new_cases_smoothed_TWN', 'new_cases_smoothed_OWID_EUR', 'avgas_cpc', 'export_kgm_weight_37079090', 'avg_board_bys', 'people_fully_vaccinated_per_hundred_USA', 'import_kgm_weight_37071000', 'change_rate_bys', 'export_kgm_weight_2804', 'export_usd_value_2804', 'export_usd_value_848620', 'import_usd_value_37079090', 'import_kgm_weight_848610', 'export_usd_value_37079090', 'export_kgm_weight_2801', 'under400_rate_bys', 'debt_rate', 'N1NG', 'export_usd_value_2801', 'export_kgm_weight_848610', 'oil_avg_value', 'import_usd_value_848610', 'import_kgm_weight_381800', 'export_kgm_weight_381800', 'import_usd_value_2801', 'import_usd_value_381800']


In [54]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=numCols, outputCol='features')

# Consolidate predictor columns
data_assembled = assembler.transform(data)

# Check the resulting column
data_assembled.select('features', 'label').show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |l

In [55]:
data_assembled.show(5)

+-----+------------+------------+------------------+----------------+-------------------+---------------+--------------------+------------+-----------------------------+------------+---------------+---------------+----------------------+----------------------+---------------------------+-----------+--------------------------+-------------+---------------------------------------+--------------------------+---------------+----------------------+---------------------+-----------------------+-------------------------+------------------------+-------------------------+----------------------+-----------------+------------+------------+---------------------+------------------------+-------------+-----------------------+------------------------+------------------------+---------------------+-----------------------+--------------------+
|label|    roa_rate|    roe_rate|cash_reinvest_rate|foreign_rate_bys|revenue_growth_rate|net_profit_rate|operating_gross_rate|current_rate|export_kgm_weight_370

In [56]:
# Split into training and testing sets in a 80:20 ratio
data_train, data_test = data_assembled.randomSplit([0.8, 0.2], seed=17)

# Check that training set has around 80% of records
training_ratio = data_train.count() / data.count()
print(training_ratio)

0.7858450274557657


# 1. DecisionTree

In [57]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(data_train)

#label is y(ROE)
#features are all of X
# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(data_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|0    |0.0       |[0.8208333333333333,0.17916666666666667]|
|0    |0.0       |[0.8208333333333333,0.17916666666666667]|
|0    |0.0       |[0.9681528662420382,0.03184713375796178]|
|0    |0.0       |[0.8208333333333333,0.17916666666666667]|
|0    |0.0       |[0.8208333333333333,0.17916666666666667]|
+-----+----------+----------------------------------------+
only showing top 5 rows



In [58]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   25|
|    0|       0.0|  110|
|    1|       1.0|  172|
|    0|       1.0|   44|
+-----+----------+-----+

0.8034188034188035


# 2.Logistic Regression

In [59]:
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression

# Create a classifier object and train on training data
logistic = LogisticRegression().fit(data_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(data_test)
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   25|
|    0|       0.0|  123|
|    1|       1.0|  172|
|    0|       1.0|   31|
+-----+----------+-----+



In [60]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: "areaUnderROC"})

precision = 0.80
recall    = 0.87


# 3. Random Forest

In [61]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import RandomForestClassifier

# Create a classifier object and fit to the training data
tree = RandomForestClassifier()
tree_model = tree.fit(data_train)

#label is y(ROE)
#features are all of X
# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(data_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|0    |0.0       |[0.9213763038498556,0.07862369615014451]|
|0    |0.0       |[0.7050486950761966,0.29495130492380334]|
|0    |0.0       |[0.8903519190599303,0.10964808094006964]|
|0    |0.0       |[0.8717692222378772,0.12823077776212272]|
|0    |0.0       |[0.7776023474922369,0.22239765250776308]|
+-----+----------+----------------------------------------+
only showing top 5 rows



In [62]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   34|
|    0|       0.0|  121|
|    1|       1.0|  163|
|    0|       1.0|   33|
+-----+----------+-----+

0.8091168091168092
